# Whiteboard Tutorial 2


## Import packages


In [1]:
import numpy as np
from scipy.optimize import root_scalar
import datetime
import math
from functools import partial

## Question 1


$$
105 = \sum_{k=1}^{6} 5 \cdot e^{-kx} + 100e^{-6x}
$$


In [2]:
def equation(x):
    i = np.repeat(5, 6)
    t = np.arange(1, 7, 1)
    p = np.sum(i * np.exp(-x * t)) + 100 * math.exp(-6 * x)
    return p - 105
y = root_scalar(equation, x0=0.1).root
print("Bond yield:", y)

freq = 4
qy = y / freq
c = 5
t = np.arange(1, 25, 1)
p = c*np.sum(np.exp(-t*qy/freq)) + 100 * math.exp(-6 * freq * qy)
print("Bond price:", p)

Bond yield: 0.039651379706951186
Bond price: 195.18428809482504


C:\Users\1tran\AppData\Local\Temp\ipykernel_4124\3934419728.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p = np.sum(i * np.exp(-x * t)) + 100 * math.exp(-6 * x)


## Question 2


In [3]:
p = [101.8137, 97.7066, 101.2414, 89.9751, 103.4012, 99.0074, 87.2097]
c = [6.5, 3.25, 4.8, 1.5, 5, 4, 2]
df0 = []
for i in range(len(c)):
    factor = (p[i]-c[i]*np.sum(np.array(df0)))/(100+c[i])
    df0 += [factor]
bond_price = 100*df0[-1]+8*np.sum(df0)
print('Discount factor:', df0)
print('Bond price:', bond_price)

Discount factor: [0.9559971830985915, 0.9162189748661459, 0.8802935347497067, 0.845776704048555, 0.8134263620589048, 0.7823129708145422, 0.7531534170659521]
Bond price: 122.89277488021438


## Question 3


In [4]:
f = 100
c = 0.05
freq = 2
coupon = f*c/freq
interest = coupon * 1/3
print('Accrued interest:', interest)

Accrued interest: 0.8333333333333334


## Question 4


In [5]:
def df_calculator(t):
    if (int(t)==t):
        return df0[int(t)-1]
    else:
        if (t<1):
            return ((df0[math.ceil(t)-1])**(t-math.floor(t)))
        else:
            return df0[math.floor(t)-1]*((df0[math.ceil(t)-1]/df0[math.floor(t)-1])**(t-math.floor(t)))
f = 100
c = 0.06
freq = 2
T = 7
coupon = c*f/freq
t = list(np.arange(1/freq, T+1/freq, 1/freq))
p = 100*df0[-1] + sum(list(map(df_calculator,t)))*coupon
print('Bond price', p)

Bond price 111.36490991503287


## Question 5


In [6]:
df1 = []
def df_calculator(t,df):
    if (int(t)==t):
        return df[int(t)-1]
    else:
        if (t<1):
            return ((df[math.ceil(t)-1])**(t-math.floor(t)))
        else:
            return df[math.floor(t)-1]*((df[math.ceil(t)-1]/df[math.floor(t)-1])**(t-math.floor(t)))
def df_optimize(x, prev_sum, coupon, price, freq):
    t = list(np.arange(start=1/freq, stop=1+1/freq, step=1/freq))
    dfn = df1 + [x]
    new_df_calc_func = partial(df_calculator,df = dfn)
    dfsum = np.sum(np.array(list(map(new_df_calc_func,t))))
    return x*100 + coupon*dfsum + prev_sum - price
p = [101.8137, 97.7066, 101.2414, 89.9751, 103.4012, 99.0074, 87.2097]
c = [6.5, 3.25, 4.8, 1.5, 5, 4, 2]
freq = 4
for i in range(len(c)):
    coupon = c[i] / freq
    t = list(np.arange(start=1/freq, stop=i+1/freq, step=1/freq))
    new_df_calc_func = partial(df_calculator,df = df1)
    dfsum = np.sum(np.array(list(map(new_df_calc_func,t))))
    df1 = df1 + [root_scalar(df_optimize,args=(dfsum*coupon/freq, coupon, p[i], freq),x0=0.8).root]
bond_price = 100*df1[-1]+8*np.sum(df1)
print('Discount factor:', df1)
print('Bond price:', bond_price)

Discount factor: [0.9549766077438229, 0.9375907548398893, 0.9427830735284667, 0.8744632090510633, 0.9384733424971641, 0.9045019110317088, 0.8247252387836003]
Bond price: 133.49263697816573


### Question 6


In [7]:
date = datetime.date(2024, 2, 26)
mature = datetime.date(2024, 4, 22)
last_coupon = datetime.date(2023, 4, 22)
quater = datetime.date(2024, 5, 26)
df = math.exp((mature-date).days*(math.log(df1[0]))/(quater-date).days)
print(104.5*df)
print(4.5*(date-last_coupon).days/(mature-last_coupon).days)

101.54705310539747
3.8114754098360657


### Question 7


In [8]:
def df_calculator(t):
    if (int(t)==t):
        return df1[int(t)-1]
    else:
        if (t<1):
            return ((df1[math.ceil(t)-1])**(t-math.floor(t)))
        else:
            return df1[math.floor(t)-1]*((df1[math.ceil(t)-1]/df1[math.floor(t)-1])**(t-math.floor(t)))
freq = 12
t = list(np.arange(start=1/freq, stop=2+1/freq, step=1/freq))
dfsum = np.sum(np.array(list(map(df_calculator,t))))
print('Monthly payment:',1000000/dfsum)

Monthly payment: 43380.78179281635


### Question 8


$$
B(t,T_i) = e^{-rT_i}
$$

Increase 50 basis point mean $$r + 0.005$$

$$
B'(t,T_i) = e^{-(r+0.005)T_i}
=e^{-rT_i} \cdot e^{-0.005T_i}
=B(t,T_i) \cdot e^{-0.005T_i}
$$


In [9]:
def df_calculator(t):
    if (int(t)==t):
        return df1[int(t)-1]
    else:
        if (t<1):
            return ((df1[math.ceil(t)-1])**(t-math.floor(t)))
        else:
            return df1[math.floor(t)-1]*((df1[math.ceil(t)-1]/df1[math.floor(t)-1])**(t-math.floor(t)))
freq = 12
t = list(np.arange(start=1/freq, stop=2+1/freq, step=1/freq))
dfnew = np.array(list(map(df_calculator,t)))
print('Monthly payment:',1000000/np.sum(dfnew*np.exp(np.arange(1, dfnew.shape[0]+1)*0.0005)))

Monthly payment: 43112.946313669745


### Question 9


In [10]:
def df_calculator(t):
    if (int(t)==t):
        return df1[int(t)-1]
    else:
        if (t<1):
            return ((df1[math.ceil(t)-1])**(t-math.floor(t)))
        else:
            return df1[math.floor(t)-1]*((df1[math.ceil(t)-1]/df1[math.floor(t)-1])**(t-math.floor(t)))
freq = 12
t = list(np.arange(start=1/freq, stop=2+1/freq, step=1/freq))
dfnew = np.array(list(map(df_calculator,t)))

payment_pm1 = 10000*0.17/12
payment1 = np.sum(np.full(24,payment_pm1) * dfnew)

payment_pm2 = 10000*0.22/12
payment2 = np.sum(np.concatenate((np.full(6,0), np.full(18,payment_pm2))) * dfnew)

print('Payment 1:', payment1)
print('Payment 2:', payment2)
print('Switch due to lower payment')


Payment 1: 3265.654993108631
Payment 2: 3140.799862333575
Switch due to lower payment


### Question 10


In [11]:
def df_calculator(t):
    if (int(t)==t):
        return df1[int(t)-1]
    else:
        if (t<1):
            return ((df1[math.ceil(t)-1])**(t-math.floor(t)))
        else:
            return df1[math.floor(t)-1]*((df1[math.ceil(t)-1]/df1[math.floor(t)-1])**(t-math.floor(t)))
freq = 12
t = list(np.arange(start=1/freq, stop=2+1/freq, step=1/freq))
dfnew = np.array(list(map(df_calculator,t)))

def rate_change(x):
    payment_pm1 = 10000*0.17/12
    payment1 = np.sum(np.full(24,payment_pm1) * dfnew)

    payment_pm2 = 10000*x/12
    payment2 = np.sum(np.concatenate((np.full(6,0), np.full(18,payment_pm2))) * dfnew)

    return payment1-payment2

rate = root_scalar(rate_change, bracket=[0,1]).root
print('Rate:',rate)

Rate: 0.22874558391953823
